In [1]:
import argparse
import random
import numpy as np
import torch
from experiment.run import multiple_run
from utils.utils import boolean_string
import logging
import sys
from types import SimpleNamespace
import time
from continuum.continuum import continuum
from continuum.data_utils import setup_test_loader
from utils.name_match import agents
from utils.setup_elements import setup_opt, setup_architecture
from utils.utils import maybe_cuda, AverageMeter
from experiment.metrics import compute_performance, single_run_avg_end_fgt
from experiment.tune_hyperparam import tune_hyper
from types import SimpleNamespace
from utils.io import load_yaml, save_dataframe_csv, check_ram_usage
import pandas as pd
import os
import pickle
from utils.utils import maybe_cuda
import torch.nn.functional as F
import torch.nn as nn
from utils.buffer.buffer_utils import random_retrieve, get_grad_vector
import copy
from utils.buffer.buffer import Buffer, Second_Buffer

In [2]:
parameters = {
    'num_runs': 1,
    'seed': 0,
    'val_size': 0.1,
    'num_val': 3,
    'num_runs_val': 3,
    'error_analysis': False,  
    'verbose': True,  
    'store': False,  
    'save_path': './',
    'imagenet_path': './imagenet1k',
    'agent': 'SUPER',
    'update': 'random',
    'retrieve': 'random',
    'second_buffer': False,  
    'update2': 'random',
    'retrieve2': 'random',
    # 'ratio': 0.2,  
    'optimizer': 'SGD',
    'learning_rate': 0.1,
    'epoch': 1,
    'batch': 10,
    'test_batch': 10000,
    'weight_decay': 0,
    'num_tasks': 20,
    'fix_order': False,  
    'plot_sample': False, 
    'data': "cifar100",
    'cl_type': "nc",
    'ns_factor': (0.0, 0.4, 0.8, 1.2, 1.6, 2.0, 2.4, 2.8, 3.2, 3.6),
    'ns_type': 'noise',
    'ns_task': (1, 1, 2, 2, 2, 2),
    'online': True, 
    'use_momentum': True,  
    'mem_size': 10000,
    'eps_mem_batch': 5,
    'sub_eps_mem_batch': 5,
    'lambda_': 100,
    'alpha': 0.9,
    'fisher_update_after': 50,
    'subsample': 50,
    'gss_mem_strength': 10,
    'gss_batch_size': 10,
    'k': 5,
    'aser_type': "asvm",
    'n_smp_cls': 2.0,
    'stm_capacity': 1000,
    'classifier_chill': 0.01,
    'log_alpha': -300,
    'minlr': 0.0005,
    'clip': 10.,
    'mem_epoch': 70,
    'labels_trick': False,  
    'separated_softmax': False,  
    'kd_trick': False, 
    'kd_trick_star': False,  
    'review_trick': False,  
    'ncm_trick': False,  
    'mem_iters': 1,
    'min_delta': 0.,
    'patience': 0,
    'cumulative_delta': False,  
    'temp': 0.07,
    'buffer_tracker': False,  
    'warmup': 4,
    'head': 'mlp',
    'exp': 'PCR',
    'Triplet': False, 
    'top5': False,  
    'mem_bank_size': 1,
    'num_subcentroids': 4,
    'PSC': False,  
    'onlyPSC': False,  
    'gamma': 0.999,
    'buffer_lip_lambda': 0.5,
    'budget_lip_lambda': 0.5,
    'headless_init_act': "relu",
    'grad_iter_step': -2,
    'lr': 0.0001,
    'optim_wd': 0.,
    'optim_mom': 0.,
    'optim_nesterov': 0,
    'ignore_other_metrics': 0,
    'debug_mode': 0,
    'reg_weight': 0.1,
    'stable_model_update_freq': 0.70,
    'stable_model_alpha': 0.999,
    'plastic_model_alpha': 0.999,
    'ucr_max': True,  
    'save_cp': False,  
    # 'cp_name': 'checkpoint.pth',
    'cp_path': './checkpoint',
}

In [3]:
args = SimpleNamespace(**parameters)
params = args
params.cuda = torch.cuda.is_available()
np.random.seed(args.seed)
random.seed(args.seed)
torch.manual_seed(args.seed)
if params.cuda:
    torch.cuda.manual_seed(params.seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
params.trick = {'labels_trick': args.labels_trick, 'separated_softmax': args.separated_softmax,
                  'kd_trick': args.kd_trick, 'kd_trick_star': args.kd_trick_star, 'review_trick': args.review_trick,
                  'ncm_trick': args.ncm_trick}
data_continuum = continuum(params.data, params.cl_type, params)

data_continuum.new_run()
test_loaders = setup_test_loader(data_continuum.test_data(), params)

Files already downloaded and verified
Files already downloaded and verified
Task: 0, Labels:[26, 86, 2, 55, 75]
Task: 1, Labels:[93, 16, 73, 54, 95]
Task: 2, Labels:[53, 92, 78, 13, 7]
Task: 3, Labels:[30, 22, 24, 33, 8]
Task: 4, Labels:[43, 62, 3, 71, 45]
Task: 5, Labels:[48, 6, 99, 82, 76]
Task: 6, Labels:[60, 80, 90, 68, 51]
Task: 7, Labels:[27, 18, 56, 63, 74]
Task: 8, Labels:[1, 61, 42, 41, 4]
Task: 9, Labels:[15, 17, 40, 38, 5]
Task: 10, Labels:[91, 59, 0, 34, 28]
Task: 11, Labels:[50, 11, 35, 23, 52]
Task: 12, Labels:[10, 31, 66, 57, 79]
Task: 13, Labels:[85, 32, 84, 14, 89]
Task: 14, Labels:[19, 29, 49, 97, 98]
Task: 15, Labels:[69, 20, 94, 72, 77]
Task: 16, Labels:[25, 37, 81, 46, 39]
Task: 17, Labels:[65, 58, 12, 88, 70]
Task: 18, Labels:[87, 36, 21, 83, 9]
Task: 19, Labels:[96, 67, 64, 47, 44]


In [4]:
model = setup_architecture(params)
model = maybe_cuda(model, params.cuda)
checkpoint1_path = f'./checkpoint/Superposition_batch500_task20_epoch50_checkpoint_run0_batch19_epoch49.pth'
checkpoint = torch.load(checkpoint1_path)
model.load_state_dict(checkpoint['model_state_dict'])

<All keys matched successfully>

In [5]:
# Check model accuracy

model.eval()
acc_array = np.zeros(len(test_loaders))
for context_matrix in range(20):
    print("context matrix number: ", context_matrix)
    with torch.no_grad():
        for task, test_loader in enumerate(test_loaders):
            # print("here")
            acc = AverageMeter()
            for i, (batch_x, batch_y) in enumerate(test_loader):
                batch_x = maybe_cuda(batch_x, "cuda")
                batch_y = maybe_cuda(batch_y, "cuda")
                logits, _, _ = model(batch_x, context_matrix)
                _, pred_label = torch.max(logits, 1)
                correct_cnt = (pred_label == batch_y).sum().item() / batch_y.size(0)
                acc.update(correct_cnt, batch_y.size(0))
                # print(correct_cnt)
            acc_array[task] = acc.avg()
        print(acc_array)

context matrix number:  0
[0.336 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
 0.    0.    0.    0.    0.    0.    0.    0.05 ]
context matrix number:  1
[0.    0.442 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
 0.    0.    0.    0.    0.    0.    0.    0.006]
context matrix number:  2
[0.    0.    0.47  0.    0.    0.    0.    0.    0.    0.    0.    0.
 0.    0.    0.    0.    0.    0.    0.    0.052]
context matrix number:  3
[0.    0.    0.    0.634 0.    0.    0.    0.    0.    0.    0.    0.
 0.    0.    0.    0.    0.    0.    0.    0.072]
context matrix number:  4
[0.    0.    0.    0.    0.54  0.    0.    0.    0.    0.    0.    0.
 0.    0.    0.    0.    0.    0.    0.    0.054]
context matrix number:  5
[0.    0.    0.    0.    0.    0.704 0.    0.    0.    0.    0.    0.
 0.    0.    0.    0.    0.    0.    0.    0.002]
context matrix number:  6
[0.   0.   0.   0.   0.   0.   0.72 0.   0.   0.   0.   0.   0.   0.
 0.   0.   0.   0.   0.   0. 

In [11]:
with torch.no_grad():
    for task, test_loader in enumerate(test_loaders):
        # print("here")
        acc = AverageMeter()
        for i, (batch_x, batch_y) in enumerate(test_loader):
            print(i)
            batch_x = maybe_cuda(batch_x, "cuda")
            batch_y = maybe_cuda(batch_y, "cuda")
            logits, _, _ = model(batch_x, context_matrix)
            print(logits.shape)

0
torch.Size([500, 100])
0
torch.Size([500, 100])
0
torch.Size([500, 100])
0
torch.Size([500, 100])
0
torch.Size([500, 100])
0
torch.Size([500, 100])
0
torch.Size([500, 100])
0
torch.Size([500, 100])
0
torch.Size([500, 100])
0
torch.Size([500, 100])
0
torch.Size([500, 100])
0
torch.Size([500, 100])
0
torch.Size([500, 100])
0
torch.Size([500, 100])
0
torch.Size([500, 100])
0
torch.Size([500, 100])
0
torch.Size([500, 100])
0
torch.Size([500, 100])
0
torch.Size([500, 100])
0
torch.Size([500, 100])


In [6]:
#Load customized model
from models.resnet_hash_t import HashResNet18
# batch_x.shape
model2 = HashResNet18(100)
model2.load_state_dict(checkpoint['model_state_dict'])
model2 = maybe_cuda(model2, params.cuda)
# z = torch.randn(20, requires_grad=True, device='cuda')
# z = torch.randn(20, device='cuda')
z = torch.zeros(20, device='cuda')
# output, _, _ = model2(batch_x, z)


In [7]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Subset
import numpy as np
from collections import defaultdict

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5071, 0.4865, 0.4409), (0.2673, 0.2564, 0.2761))  # CIFAR-100 的标准化值
])

train_dataset = torchvision.datasets.CIFAR100(root='./datasets', train=True, download=True, transform=transform)

class_indices = defaultdict(list)
for idx, (_, label) in enumerate(train_dataset):
    class_indices[label].append(idx)

target_fraction = 1
num_classes = 100
per_class_samples = int(len(train_dataset) * target_fraction // num_classes)

selected_indices = []
for label, indices in class_indices.items():
    selected_indices.extend(np.random.choice(indices, per_class_samples, replace=False))

subset_dataset = Subset(train_dataset, selected_indices)

batch_size = 1000
sub_train_loader = DataLoader(subset_dataset, batch_size=batch_size, shuffle=True, num_workers=4)

Files already downloaded and verified


In [8]:
from tqdm import tqdm

z_o = torch.randn(20, requires_grad=True, device='cuda')
for param in model2.parameters():
    param.requires_grad = False
optimizer = torch.optim.SGD([z], lr=0.1)
loss_fn = torch.nn.CrossEntropyLoss(reduction='mean')

for _ in tqdm(range(5)):
    for x, y in sub_train_loader:
        optimizer.zero_grad()
        x = maybe_cuda(x, "cuda")
        y = maybe_cuda(y, "cuda")
        output, _, _ = model2(x, z_o)
        loss = loss_fn(output, y)
        loss.backward()
        optimizer.step()

  0%|                                                                                            | 0/5 [00:09<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
batch_x.shape

In [ ]:
print(z_o)

In [ ]:

model2.eval()
acc_array = np.zeros(len(test_loaders))
with torch.no_grad():
    for task, test_loader in enumerate(test_loaders):
        # print("here")
        acc = AverageMeter()
        for i, (batch_x, batch_y) in enumerate(test_loader):
            batch_x = maybe_cuda(batch_x, "cuda")
            batch_y = maybe_cuda(batch_y, "cuda")
            logits, _, _ = model2(batch_x, z_o)
            _, pred_label = torch.max(logits, 1)
            correct_cnt = (pred_label == batch_y).sum().item() / batch_y.size(0)
            acc.update(correct_cnt, batch_y.size(0))
            # print(correct_cnt)
        acc_array[task] = acc.avg()
    print(acc_array)

In [9]:
model2.eval()
acc_array = np.zeros(len(test_loaders))
with torch.no_grad():
    for task, test_loader in enumerate(test_loaders):
        # print("here")
        acc = AverageMeter()
        for i, (batch_x, batch_y) in enumerate(test_loader):
            batch_x = maybe_cuda(batch_x, "cuda")
            batch_y = maybe_cuda(batch_y, "cuda")
            z0 = torch.zeros(20, device='cuda')
            z0[task] = 1
            logits, _, _ = model2(batch_x, z0)
            _, pred_label = torch.max(logits, 1)
            correct_cnt = (pred_label == batch_y).sum().item() / batch_y.size(0)
            acc.update(correct_cnt, batch_y.size(0))
            # print(correct_cnt)
        acc_array[task] = acc.avg()
    print(acc_array)

[0.338 0.442 0.47  0.634 0.54  0.704 0.72  0.586 0.712 0.756 0.778 0.698
 0.716 0.782 0.746 0.844 0.774 0.848 0.882 0.748]


In [26]:
import torch
import torch.nn.functional as F

# Assuming context_matrices is the total number of possible context matrices
num_context_matrices = 20
context_matrix_counts = torch.zeros(len(test_loaders), num_context_matrices)  # To store the count of each context matrix for every task

with torch.no_grad():
    for task, test_loader in enumerate(test_loaders):
        for i, (batch_x, batch_y) in enumerate(test_loader):
            batch_x = maybe_cuda(batch_x, "cuda")
            batch_y = maybe_cuda(batch_y, "cuda")

            # Expand batch_x to accommodate the number of context_matrices
            batch_size = batch_x.size(0)
            expanded_batch_x = batch_x.unsqueeze(1).repeat(1, num_context_matrices, 1, 1, 1)
            
            # Store logits for all context_matrices
            all_logits = torch.zeros(batch_size, num_context_matrices, 100).cuda()
            
            # Compute logits for each context_matrix and store them
            for context_idx in range(num_context_matrices):
                logits, _, _ = model(expanded_batch_x[:, context_idx], context_idx)
                all_logits[:, context_idx, :] = logits

            # Find the maximum logit for each sample across all context_matrices
            max_logits, best_context_indices = torch.max(all_logits.max(dim=-1)[0], dim=1)
            
            # Count the occurrences of the best context matrix
            for idx in range(batch_size):
                context_matrix_counts[task, best_context_indices[idx]] += 1

# Print the context matrix count for each task
print(context_matrix_counts)


tensor([[  0.,   0.,   0.,   0.,   0.,   0.,   1.,   0.,   1.,   1.,   2.,  16.,
           4.,   7.,  10.,  25.,  22.,  22., 118., 271.],
        [  0.,   0.,   0.,   0.,   1.,   0.,   0.,   0.,   5.,   2.,   7.,  25.,
           3.,   3.,   8.,  41.,  12.,  51.,  92., 250.],
        [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   1.,   6.,   0.,   5.,  11.,
           5.,   5.,   8.,   6.,  29.,  81., 126., 217.],
        [  0.,   0.,   0.,   0.,   0.,   0.,   1.,   0.,   4.,   2.,   4.,  16.,
           3.,   7.,  16.,  19.,  14.,  39.,  90., 285.],
        [  0.,   0.,   0.,   0.,   0.,   0.,   8.,   0.,   2.,   0.,   4.,  10.,
           4.,   9.,  13.,  13.,  15.,  80., 146., 196.],
        [  0.,   0.,   0.,   0.,   0.,   1.,   0.,   0.,   3.,   0.,   6.,   5.,
           3.,   7.,  13.,  16.,  15.,  45., 120., 266.],
        [  0.,   0.,   0.,   0.,   1.,   1.,  27.,   0.,   1.,   1.,   1.,   7.,
           1.,   5.,  11.,  13.,  20.,  74.,  86., 251.],
        [  0.,   0.,   0., 

In [17]:
context_matrix_counts[0]

tensor([  0.,   0.,   0.,   0.,   0.,   0.,   1.,   0.,   1.,   1.,   2.,  16.,
          4.,   7.,  10.,  25.,  22.,  22., 118., 271.])

In [22]:
import torch
import torch.nn.functional as F

# Assuming context_matrices is the total number of possible context matrices
num_context_matrices = 20
temperature = 2.0  # You can adjust this for more or less smoothing
context_matrix_counts = torch.zeros(len(test_loaders), num_context_matrices)  # To store the count of each context matrix for every task

def smooth_and_project(logits, temp=10, range_min=0.0, range_max=1.0):
    # Step 1: Apply temperature scaling to smooth the logits
    smoothed_logits = F.softmax(logits / temp, dim=-1)  # Apply softmax with temperature scaling

    # Step 2: Normalize the logits to the [range_min, range_max] range
    min_val = smoothed_logits.min(dim=-1, keepdim=True)[0]  # Get the minimum value
    max_val = smoothed_logits.max(dim=-1, keepdim=True)[0]  # Get the maximum value
    projected_logits = (smoothed_logits - min_val) / (max_val - min_val)  # Min-max normalization
    projected_logits = projected_logits * (range_max - range_min) + range_min  # Scale to the desired range

    return projected_logits

with torch.no_grad():
    for task, test_loader in enumerate(test_loaders):
        for i, (batch_x, batch_y) in enumerate(test_loader):
            batch_x = maybe_cuda(batch_x, "cuda")
            batch_y = maybe_cuda(batch_y, "cuda")

            # Expand batch_x to accommodate the number of context_matrices
            batch_size = batch_x.size(0)
            expanded_batch_x = batch_x.unsqueeze(1).repeat(1, num_context_matrices, 1, 1, 1)
            
            # Store logits for all context_matrices
            all_logits = torch.zeros(batch_size, num_context_matrices, 100).cuda()
            
            # Compute logits for each context_matrix and store them
            for context_idx in range(num_context_matrices):
                logits, _, _ = model(expanded_batch_x[:, context_idx], context_idx)
                all_logits[:, context_idx, :] = logits

            # Step 3: Smooth and project logits before proceeding
            smoothed_logits = smooth_and_project(all_logits, temp=temperature)

            # Step 4: Find the maximum smoothed logit for each sample across all context_matrices
            max_logits, best_context_indices = torch.max(smoothed_logits.max(dim=-1)[0], dim=1)
            
            # Count the occurrences of the best context matrix
            for idx in range(batch_size):
                context_matrix_counts[task, best_context_indices[idx]] += 1

# Print the context matrix count for each task
print(context_matrix_counts)



tensor([[500.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
           0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.],
        [500.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
           0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.],
        [500.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
           0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.],
        [500.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
           0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.],
        [500.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
           0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.],
        [500.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
           0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.],
        [500.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
           0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.],
        [500.,   0.,   0., 

In [30]:
checkpoint2_path = f'./checkpoint/Superposition_batch500_task10_epoch1_mce_checkpoint_run0_batch5_epoch0.pth'
checkpoint = torch.load(checkpoint2_path)
model.load_state_dict(checkpoint['model_state_dict'])
num_context_matrices = 5
context_matrix_counts = torch.zeros(len(test_loaders), num_context_matrices)  # To store the count of each context matrix for every task

with torch.no_grad():
    for task, test_loader in enumerate(test_loaders):
        for i, (batch_x, batch_y) in enumerate(test_loader):
            batch_x = maybe_cuda(batch_x, "cuda")
            batch_y = maybe_cuda(batch_y, "cuda")

            # Expand batch_x to accommodate the number of context_matrices
            batch_size = batch_x.size(0)
            expanded_batch_x = batch_x.unsqueeze(1).repeat(1, num_context_matrices, 1, 1, 1)
            
            # Store logits for all context_matrices
            all_logits = torch.zeros(batch_size, num_context_matrices, 100).cuda()
            
            # Compute logits for each context_matrix and store them
            for context_idx in range(num_context_matrices):
                logits, _, _ = model(expanded_batch_x[:, context_idx], context_idx)
                all_logits[:, context_idx, :] = logits

            # Find the maximum logit for each sample across all context_matrices
            max_logits, best_context_indices = torch.max(all_logits.max(dim=-1)[0], dim=1)
            
            # Count the occurrences of the best context matrix
            for idx in range(batch_size):
                context_matrix_counts[task, best_context_indices[idx]] += 1

# Print the context matrix count for each task
print(context_matrix_counts)


tensor([[ 14.,  37.,  38.,  61., 350.],
        [ 51.,  34.,  48.,  83., 284.],
        [ 27.,  55.,  42.,  75., 301.],
        [ 17.,  37.,  40.,  90., 316.],
        [ 27.,  42.,  55.,  97., 279.],
        [ 24.,  44.,  65.,  84., 283.],
        [  6.,  19.,  27., 120., 328.],
        [  8.,  26.,  35.,  91., 340.],
        [ 10.,  29.,  41.,  59., 361.],
        [  8.,  29.,  41.,  73., 349.],
        [ 24.,  32.,  39.,  52., 353.],
        [ 15.,  37.,  46.,  87., 315.],
        [ 13.,  30.,  49.,  83., 325.],
        [  4.,  19.,  28.,  83., 366.],
        [ 10.,  24.,  25.,  89., 352.],
        [ 16.,  43.,  44.,  77., 320.],
        [  8.,  34.,  30.,  76., 352.],
        [ 30.,  38.,  26.,  91., 315.],
        [ 15.,  35.,  53.,  55., 342.],
        [ 14.,  42.,  33.,  95., 316.]])


In [31]:
checkpoint2_path = f'./checkpoint/Superposition_batch500_task10_epoch50_smooth_checkpoint_run0_batch5_epoch49.pth'
checkpoint = torch.load(checkpoint2_path)
model.load_state_dict(checkpoint['model_state_dict'])
num_context_matrices = 10
context_matrix_counts = torch.zeros(len(test_loaders), num_context_matrices)  # To store the count of each context matrix for every task

with torch.no_grad():
    for task, test_loader in enumerate(test_loaders):
        for i, (batch_x, batch_y) in enumerate(test_loader):
            batch_x = maybe_cuda(batch_x, "cuda")
            batch_y = maybe_cuda(batch_y, "cuda")

            # Expand batch_x to accommodate the number of context_matrices
            batch_size = batch_x.size(0)
            expanded_batch_x = batch_x.unsqueeze(1).repeat(1, num_context_matrices, 1, 1, 1)
            
            # Store logits for all context_matrices
            all_logits = torch.zeros(batch_size, num_context_matrices, 100).cuda()
            
            # Compute logits for each context_matrix and store them
            for context_idx in range(num_context_matrices):
                logits, _, _ = model(expanded_batch_x[:, context_idx], context_idx)
                all_logits[:, context_idx, :] = logits

            # Find the maximum logit for each sample across all context_matrices
            max_logits, best_context_indices = torch.max(all_logits.max(dim=-1)[0], dim=1)
            
            # Count the occurrences of the best context matrix
            for idx in range(batch_size):
                context_matrix_counts[task, best_context_indices[idx]] += 1

# Print the context matrix count for each task
print(context_matrix_counts)

tensor([[ 65.,  78.,  69.,  83., 205.],
        [ 81.,  88.,  68., 115., 148.],
        [ 25., 141.,  73.,  95., 166.],
        [ 33., 117.,  48.,  91., 211.],
        [ 22.,  42., 135., 142., 159.],
        [ 21.,  54., 138.,  94., 193.],
        [ 13.,  36.,  36., 252., 163.],
        [ 15.,  50.,  43., 206., 186.],
        [ 18.,  43.,  36.,  47., 356.],
        [ 24.,  41.,  37.,  88., 310.],
        [ 29.,  66.,  61., 114., 230.],
        [ 26.,  66.,  53., 165., 190.],
        [ 26.,  79.,  63., 123., 209.],
        [ 21.,  66.,  55., 124., 234.],
        [ 33.,  48.,  63., 153., 203.],
        [ 34.,  96.,  61., 112., 197.],
        [ 39.,  65.,  49., 129., 218.],
        [ 31.,  97.,  66., 106., 200.],
        [ 42.,  72.,  73., 109., 204.],
        [ 42.,  70.,  67., 140., 181.]])
